<a href="https://colab.research.google.com/github/steinhaug/stable-diffusion/blob/main/tool/drive_tools.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Quick colab for Drive related tasks, and tasks not requiring GPU typically. Loads config from drive so that notebook can be added to github.

In [ ]:
def parse_transcript(file_path, parse_mode, save_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()

    filtered_lines = []
    for line in lines:
        stripped_line = line.strip()
        if parse_mode == "text":
            if stripped_line and not stripped_line.replace(':', '').isdigit():
                filtered_lines.append(stripped_line)
        elif parse_mode == "timecode":
            if stripped_line and stripped_line.replace(':', '').isdigit():
                filtered_lines.append(stripped_line)

    with open(save_path, 'w') as file:
        for line in filtered_lines:
            file.write(line + '\n')

# Example usage:

#parse_transcript('2156.txt', 'text', '2156-txt.txt')
# parse_transcript('file.txt', 'timecode', 'saved_timecodes.txt')


In [11]:
#@title Load settings from drive config file

# Load config from drive
KISTA_DRIVE_PATHS = "/content/drive/MyDrive/docs"
INI_FILE_LOC = f"{KISTA_DRIVE_PATHS}/config_SDTools.ini"

# Make sure gDrive is loaded
inip = Path("/content/drive/MyDrive")
if not inip.is_dir():
    from google.colab import drive
    drive.mount('/content/drive')

import os
from pathlib import Path
from IPython.display import display, Markdown, clear_output
from ipywidgets import widgets, GridspecLayout, Layout
from configparser import ConfigParser

def return__isValidDir(directory):
    if os.path.isdir(directory):
        return True
    else:
        return False

def strtobool (val):
    """Quick cast tro boolean
    """
    val = val.lower()
    if val in ('y', 'yes', 't', 'true', 'on', '1'):
        return True
    elif val in ('', 'n', 'no', 'f', 'false', 'off', '0'):
        return False
    else:
        raise ValueError("invalid truth value %r" % (val,))

def getValue (savedconf, key, type):
    try:
        val = savedconf[key]
    except KeyError:
        val = ""
    if type == 'bool':
        return strtobool(val)
    elif type == 'int':
        return int(val)
    else:
        return val

def update_KISTA_config():
    config_object = ConfigParser()
    config_object["CONF"] = {
        "hf_read_token": hf_read_token.value,
        "civitai_token": civitai_token.value,
        "sd15_models": sd15_models.value,
        "sdxl_models": sdxl_models.value,
    }
    with open(INI_FILE_LOC, 'w') as conf:
        config_object.write(conf)

def btn_continue_event_handler(btn_object):
    grid.close();
    update_KISTA_config()
    clear_output()
    print('Settings saved! ✓')


# Setup variables
hf_read_token = widgets.Text( value="", placeholder='Huggingface Read Token', disabled=False )
civitai_token = widgets.Text( value="", placeholder='civitai API Token', disabled=False )


sd15_models = widgets.Text( value="", placeholder='SD v1.5 models folder', disabled=False )
sdxl_models = widgets.Text( value="", placeholder='SDXL models folder', disabled=False )

# Setup headers
btn_continue = widgets.Button(description='Save configuration and continue', button_style='info', layout=Layout(width='auto', height='auto'))
btn_continue.on_click(btn_continue_event_handler)
btn1 = widgets.Button(description='KEYS', layout=Layout(width='auto', height='auto'))
btn1.style.button_color='White'; btn1.style.font_weight='bold';
btn2 = widgets.Button(description='GOOGLE DRIVE PATHS', layout=Layout(width='auto', height='auto'))
btn2.style.button_color='White'; btn2.style.font_weight='bold';

# Setup grid
grid = GridspecLayout(4, 2)
grid[0, 0] = btn1
grid[1, 0] = hf_read_token
grid[2, 0] = civitai_token
grid[0, 1] = btn2
grid[1, 1] = sd15_models
grid[2, 1] = sdxl_models
grid[3, :] = btn_continue
display(grid)

# Init config
inip = Path(INI_FILE_LOC)
if inip.is_file():
    config_object = ConfigParser()
    config_object.read(INI_FILE_LOC)
    savedconf = config_object["CONF"]
    hf_read_token.value = getValue(savedconf, 'hf_read_token', 'string')
    civitai_token.value = getValue(savedconf, 'civitai_token', 'string')
    sd15_models.value = getValue(savedconf, 'sd15_models', 'string')
    sdxl_models.value = getValue(savedconf, 'sdxl_models', 'string')
else:
    if not return__isValidDir(KISTA_DRIVE_PATHS):
        os.makedirs(KISTA_DRIVE_PATHS)


In [ ]:
# misc system settings
!lscpu
!nvidia-smi -q
!nvidia-smi -L